# _*Experiment with the Simon's Algorithm in Aqua*_

This notebook demonstrates how to experiment with the `Simon`'s algorithm in `Qiskit Aqua`.

We first import all necessary modules.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua import run_algorithm
from qiskit.aqua.algorithms import Simon
from qiskit.aqua.components.oracles import TruthTableOracle

The [Simon's algorithm](https://en.wikipedia.org/wiki/Simon's_problem) is explained in more detail in the corresponding notebook located in the directory `community/algorithms`. We can experiment with it in Aqua by feeding it oracles created using truth tables. For example, we can create a `TruthTableOracle` instance as follows.

In [2]:
bitmaps = [
    '01101001', 
    '10011001', 
    '01100110'
]
oracle = TruthTableOracle(bitmaps)

As shown, the truthtable is specified with three length-8 bitstrings, each containing the values of all entries for a particular output column in the table. Each bitstring has length $8$, so the truthtable has $3$ input bits; There are $3$ bitstrings, so the truthtable has $3$ output bits.

The function $f$ represented by the truthtable is promised to be either 1-to-1 or 2-to-1. Our goal is to determine which. For the case of 2-to-1, we also need to compute the mask $\mathbf{s}$, which satisfies $\forall \mathbf{x},\mathbf{y}$: $\mathbf{x} \oplus \mathbf{y} = \mathbf{s}$ iff $f(\mathbf{x}) = f(\mathbf{y})$. Apparently, if $f$ is 1-to-1, the corresponding mask $\mathbf{s} = \mathbf{0}$.

Let us first compute the groundtruth mask $\mathbf{s}$ classically:

In [3]:
def compute_mask(input_bitmaps):
    vals = list(zip(*input_bitmaps))[::-1]
    def find_pair():
        for i in range(len(vals)):
            for j in range(i + 1, len(vals)):
                if vals[i] == vals[j]:
                    return i, j
        return 0, 0

    k1, k2 = find_pair()
    return np.binary_repr(k1 ^ k2, int(np.log2(len(input_bitmaps[0]))))

mask = compute_mask(bitmaps)
print(f'The groundtruth mask is {mask}.')

The groundtruth mask is 011.


Next we can create a `Simon` instance using the oracle, and run it to check the result against the groundtruth.

In [4]:
simon = Simon(oracle)
backend = BasicAer.get_backend('qasm_simulator')
result = simon.run(QuantumInstance(backend, shots=1024))
print('The mask computed using Simon is {}.'.format(result['result']))
assert(result['result'] == mask)

The mask computed using Simon is 011.


The above step-by-step programatic approach can also be achieved by using a json configuration dictionary with the parameters for the algorithm and any other dependent objects it requires, as follows:

In [5]:
params = {
    'problem': {
        'name': 'periodfinding',
    },
    'algorithm': {
        'name': 'Simon'
    },
    'oracle': {
        'name': 'TruthTableOracle',
        'bitmaps': bitmaps
    },
    'backend': {
        'shots': 1024,
    },
}

result_dict = run_algorithm(params, backend=backend)
print('The mask computed using Simon is {}.'.format(result_dict['result']))
assert(result_dict['result'] == mask)

The mask computed using Simon is 011.


We can also quickly try a truthtable that represents a 1-to-1 function (i.e., the corresponding mask is $\mathbf{0}$), as follows.

In [6]:
bitmaps = [
    '00011110', 
    '01100110', 
    '10101010'
]
mask = compute_mask(bitmaps)
print(f'The groundtruth mask is {mask}.')
oracle = TruthTableOracle(bitmaps)
simon = Simon(oracle)
result = simon.run(QuantumInstance(backend, shots=1024))
print('The mask computed using Simon is {}.'.format(result['result']))
assert(result['result'] == mask)

The groundtruth mask is 000.
The mask computed using Simon is 000.
